In [7]:
import pandas as pd
import numpy as np
import os
import folium
from folium import plugins
import webbrowser
import chardet
from datetime import datetime

# rawdata = ""
# with open('2021_MCM_Problem_C_Data/2021MCMProblemC_DataSet.xlsx', 'r') as f:
#     rawdata += f.readline()

# chardet.detect(rawdata)

data = pd.read_excel("2021_MCM_Problem_C_Data/2021MCMProblemC_DataSet.xlsx", datetime = "Detection Date")

for i in range(0, len(data)) :
    if(isinstance(data.iloc[i]["Detection Date"], datetime)) : 
        data.loc[i, "Detection Date"] = data.iloc[i]["Detection Date"].year
    else :
        data.loc[i, "Detection Date"] = np.nan

PosData2020 = data[(data["Detection Date"] == 2020) & (data["Lab Status"] == "Positive ID")]

PosData2019 = data[(data["Detection Date"] == 2019) & (data["Lab Status"] == "Positive ID")]

In [8]:
print("2019: {0} : {1}".format(PosData2019["Latitude"].mean(), PosData2019["Longitude"].mean()))

print("2020: {0} : {1}".format(PosData2020["Latitude"].mean(), PosData2020["Longitude"].mean()))

2019: 49.024412000000005 : -122.96909459999999
2020: 48.959598666666665 : -122.59492977777775


In [28]:
import matplotlib.pyplot as plt
from bokeh.plotting import figure, save
from bokeh.models import GeoJSONDataSource
import webbrowser
import json


with open("us-all-mainland.geo.json", encoding="utf8") as f:
    temp_data = json.loads(f.read())

def wgs84_to_web_mercator(df, lon="Longitude", lat="Latitude"):

    k = 6378137
    df.loc[:, "x"] = df[lon] * (k * np.pi/180.0)
    df.loc[:, "y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k

    return df

PosData2020 = wgs84_to_web_mercator(PosData2020)

print(PosData2020["x"])

def isInDC(district):
    if 'woe-name' in district['properties'] and 'Washington' in district['properties']['woe-name']:
        return True
    
    return False

temp_data['features'] = list(filter(isInDC, temp_data['features']))

DC_geo_source = GeoJSONDataSource(geojson=json.dumps(temp_data))

p = figure(width=500, height=500)

p.scatter(PosData2020["x"], PosData2020["y"])

p.patches(xs='xs', ys='ys', source=DC_geo_source)

save(obj=p, filename="DC_Map.html")

webbrowser.open("DC_Map.html")

834    -1.365456e+07
956    -1.365241e+07
1011   -1.362758e+07
3279   -1.366391e+07
4127   -1.364497e+07
4206   -1.364569e+07
4207   -1.364582e+07
4208   -1.364496e+07
4338   -1.364496e+07
x                NaN
y                NaN
Name: x, dtype: float64


C:\Users\uzuki\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\uzuki\anaconda3\lib\site-packages\bokeh\io\saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\Users\uzuki\anaconda3\lib\site-packages\bokeh\io\saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh

True